# Blink Features from MNE Epochs

This notebook shows how to compute blink features every 30 seconds when your data is already segmented into `mne.Epochs`. We still rely on the sample `ear_eog.fif` file.

In [ ]:
from pathlib import Path
import mne
from tqdm import tqdm

from pyear.pipeline import extract_features

## 1. Create Epochs from the raw recording

In [ ]:
fif_path = Path("../unitest/ear_eog.fif")
raw = mne.io.read_raw_fif(fif_path, preload=True)
events = mne.make_fixed_length_events(raw, id=1, duration=30.0)
epochs = mne.Epochs(raw, events, tmin=0.0, tmax=30.0 - 1.0 / raw.info["sfreq"], baseline=None, preload=True)

## 2. Convert blink annotations to dictionaries

In [ ]:
sfreq = raw.info["sfreq"]
epoch_len = 30.0
n_epochs = len(epochs)
ann = raw.annotations
blinks = []
start_times = epochs.events[:, 0] / sfreq + epochs.tmin
for idx, start in enumerate(start_times):
    stop = start + epoch_len
    signal = epochs.get_data(picks="EAR-avg_ear")[idx, 0]
    for onset, dur, desc in zip(ann.onset, ann.duration, ann.description):
        if desc != 'blink':
            continue
        if onset >= start and onset + dur <= stop:
            s = int((onset - start) * sfreq)
            e = int((onset + dur - start) * sfreq)
            blinks.append({'refined_start_frame': s,
                           'refined_peak_frame': (s + e) // 2,
                           'refined_end_frame': e,
                           'epoch_signal': signal,
                           'epoch_index': idx})

## 3. Compute features

In [ ]:
df = extract_features(blinks, sfreq, epoch_len, n_epochs)
df.head()